In [2]:
import numpy as np
import matplotlib as plt

#Problem a: Phasefactor matrix

def P(ni,Di,k0):
    Phasetransfer = np.zeros((2,2),dtype = 'complex');
    Phasetransfer[0,0] = np.exp(1j*k0*Di*ni)
    Phasetransfer[1,1] = np.exp(-1j*k0*Di*ni)
    return Phasetransfer

k0 = 2*np.pi/(5.5e-7); D = 0.01; n = 1.5
A = P(n,D,k0);
print(A)

[[-0.14231484-0.98982144j  0.        +0.j        ]
 [ 0.        +0.j         -0.14231484+0.98982144j]]
